# Задача о планировании тренировок 
# (в первом приближении)

Модель представляет из себя расписание тренировок для команды.

#### __Мотивация__: 

в студенческих командах зачастую ограничены ресурсы, а именно:
- имеется ограниченное количество тренировок (слишком малое)
- игрокам не позволяет расписание всегда присутствовать на всех тренировках
Существующие тренировочные программы рассчитаны на больший ресурс, поэтому в описанном положении становятся менее эффективными

#### __Решение__: 

составить такое расписание, которое будет учитывать расписание игроков и их индивидуальные потребности в развитии по
трем направлениям: 
- техника 
- физическая подготовка 
- командные взаимодействия

Таким образом, расписание должно расставлять упражнения в том порядке, в котором они принесут наибольшую пользу игрокам.

В качестве *параметров* модели обсуждаются вместе с игроками и тренером их потребности в различных упражнениях в процентном соотношении, а также собственно то, насколько различные упражнения закрывают ту или иную потребность

В качестве *переменных* определяется собственно расписание упражнений

В качестве *целевой функции* нужно выбрать какую-то общую характеристику, основывающуюся на закрытии потребностей всех игроков по отдельности.



#### Множества индексов:

I = {1, ..., I} - игроки

J = {1, ..., J} - упражнения

K = {1, ..., T} - минуты в течение тренировочного плана (длительность тренировки * количество)

#### Параметры:

(в процентах)

0 <= a[i] <= 1 - потребность игрока i в упражнениях на технику 

0 <= b[i] <= 1 - потребность игрока i в упражнениях на физику

0 <= c[i] <= 1 - потребность игрока i в командных упражнениях

a[i]+b[i]+c[i] = 1

d[i][k] - способность игрока i присутствовать в k-ую минуту тренировочного процесса

0 <= aa[j] <= 1 - удовлетворение потребности в упражнениях на технику j-ым упражнением

0 <= bb[j] <= 1 - удовлетворение потребности в упражнениях на физику j-ым упражнением

0 <= cc[j] <= 1 - удовлетворение потребности в командных упражнениях j-ым упражнением

aa[j]+bb[j]+cc[j] = 1

t[j] - длительность j-го упражнения

#### Переменные:

x[k][j] = 1 - если в минуту k идет упражнение j, 0 - иначе

y[k][j] = 1 - если в минуту k начинается упражнение j, 0 - иначе


Нужно поиграться с ограничениями, чтобы можно было отказаться либо от y, либо от x

#### Целевая функция:

Для начала вычисляем максимальную возможную реализацию всех потребностей

fa*[i] = сумма_по_k_от_1_до_T(d[i][k]) * a[i] - максимальное возможное удовлетворение потребности игрока i в упражнениях на технику

fb*[i] = сумма_по_k_от_1_до_T(d[i][k]) * b[i] - максимальное возможное удовлетворение потребности игрока i в упражнениях на физику

fc*[i] = сумма_по_k_от_1_до_T(d[i][k]) * c[i] - максимальное возможное удовлетворение потребности игрока i в упражнениях на командные взаимодействия

Далее посчитаем, как модель удовлетворяет потребности на самом деле:

fa[i] = сумма_по_k_от_1_до_T(d[i][k] * сумма_по_j_от_1_до_J(x[k][j]\*aa[j])) / fa\*[i] - реальное удовлетворение потребности игрока i в упражнениях на технику

fb[i] = сумма_по_k_от_1_до_T(d[i][k] * сумма_по_j_от_1_до_J(x[k][j]\*bb[j])) / fb\*[i] - реальное удовлетворение потребности игрока i в упражнениях на физику

fc[i] = сумма_по_k_от_1_до_T(d[i][k] * сумма_по_j_от_1_до_J(x[k][j]\*cc[j])) / fc\*[i] - реальное удовлетворение потребности игрока i в упражнениях на командные взаимодействия

Целевая функция должна как-то агрегировать эти значения, пока что будет усреднять:

f = сумма_по_i_от_1_до_I(fa[i]+fb[i]+fc[i])

### Установка зависимостей (делается один раз, потом иногда можно обновлять, но необязательно)

In [ ]:
using Pkg

Pkg.update("JuMP")
Pkg.add("HiGHS")
Pkg.add("EzXML")

    Updating registry at `C:\Users\romag\.julia\registries\General.toml`


### Инициализация модели

In [ ]:
using JuMP
using HiGHS
using EzXML

model = Model(HiGHS.Optimizer)

### Константы

In [ ]:
D = 120 # Duration - Длительность тренировки в минутах
P = 24 # Plan - количество тренировок (24 тренировки - 2 месяца)

T = D*P # Training plan - горизонт планирования

### Множества индексов

In [ ]:
I = 10 # Количество игроков
J = 10 # Количество упражнений
K = T # Количество минут в тренировочном плане

### Оптимизационные переменные

In [ ]:
@variable(model, x[k = 1:T, j = 1:J], binary = true)

In [ ]:
@variable(model, y[k = 1:T, j = 1:J], binary = true)

### Параметры модели

In [69]:
a = zeros(Float16, I)
for i in 1:I
    a[i] = 0.33
end

b = zeros(Float16, I)
for i in 1:I
    b[i] = 0.33
end

c = zeros(Float16, I)
for i in 1:I
    c[i] = 0.33
end

d = zeros(Int, I, T)
for i in 1:I
    for k in 1:T
        d[i, k] = 1
    end
end

aa = zeros(Float16, J)
for j in 1:J
    aa[j] = 0.33
end

bb = zeros(Float16, J)
for j in 1:J
    bb[j] = 0.33
end

cc = zeros(Float16, J)
for j in 1:J
    cc[j] = 0.33
end


t = zeros(Int, J)
for j in 1:J
    t[j] = 15
end

### Ограничения

In [ ]:
@constraint(model, [k = 1:T], sum(x[k, j] for j in 1:J) <= 1) #1_1 - в каждую минуту идет только одно упражнение

@constraint(model, [k = 1:T], sum(y[k, j] for j in 1:J) <= 1) #1_2 - в каждую минуту начинается только одно упражнение

# @constraint(model, ) #2 упражнение можно выполнить только целиком, т.е. если оно началось, то идет непрерывно ровно столько, сколько оно длится
# это ограничение очень важное, но пока непонятно, как записать его в линейном формате без использования одновременно и x и y

@constraint(model, [k = 1:T, j = 1:J], y[k, j] + t[j] <= T) #3 упражнение не может заканчиваться позже конца тренировочного процесса

# @constraint(model, ) #4 упражнение не может начаться примерно в конце i-ой тренировки и закончиться примерно в начале i+1-ой

### Целевая функция

In [71]:
fa_opt = zeros(Float16, I)
fb_opt = zeros(Float16, I)
fc_opt = zeros(Float16, I)
for i in 1:I
    fa_opt[i] = sum(d[i, k] for k in 1:T) * a[i]
    fb_opt[i] = sum(d[i, k] for k in 1:T) * b[i]
    fc_opt[i] = sum(d[i, k] for k in 1:T) * c[i]
end

fa = zeros(AffExpr, I)
fb = zeros(AffExpr, I)
fc = zeros(AffExpr, I)
for i in 1:I
    fa[i] = sum(d[i, k] * sum(x[k, j]*aa[j] for j in 1:J) for k in 1:T) / fa_opt[i]
    fb[i] = sum(d[i, k] * sum(x[k, j]*bb[j] for j in 1:J) for k in 1:T) / fb_opt[i]
    fc[i] = sum(d[i, k] * sum(x[k, j]*cc[j] for j in 1:J) for k in 1:T) / fc_opt[i]
end

In [75]:
f = ( sum(fa) + sum(fb) + sum(fc) ) / (3*I)

@objective(model, Max, f)

0.0003472678853235139 x[1,1] + 0.0003472678853235139 x[1,2] + 0.0003472678853235139 x[1,3] + 0.0003472678853235139 x[1,4] + 0.0003472678853235139 x[1,5] + 0.0003472678853235139 x[1,6] + 0.0003472678853235139 x[1,7] + 0.0003472678853235139 x[1,8] + 0.0003472678853235139 x[1,9] + 0.0003472678853235139 x[1,10] + 0.0003472678853235139 x[2,1] + 0.0003472678853235139 x[2,2] + 0.0003472678853235139 x[2,3] + 0.0003472678853235139 x[2,4] + 0.0003472678853235139 x[2,5] + 0.0003472678853235139 x[2,6] + 0.0003472678853235139 x[2,7] + 0.0003472678853235139 x[2,8] + 0.0003472678853235139 x[2,9] + 0.0003472678853235139 x[2,10] + 0.0003472678853235139 x[3,1] + 0.0003472678853235139 x[3,2] + 0.0003472678853235139 x[3,3] + 0.0003472678853235139 x[3,4] + 0.0003472678853235139 x[3,5] + 0.0003472678853235139 x[3,6] + 0.0003472678853235139 x[3,7] + 0.0003472678853235139 x[3,8] + 0.0003472678853235139 x[3,9] + 0.0003472678853235139 x[3,10] + [[...28740 terms omitted...]] + 0.0003472678853235139 x[2878,1] + 0

In [76]:
optimize!(model)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [3e-04, 3e-04]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 3e+03]
Presolving model
2880 rows, 28800 cols, 28800 nonzeros  0s
0 rows, 2880 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Solving report
  Status            Optimal
  Primal bound      1.00013150973
  Dual bound        1.00013150973
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    1.00013150973 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.05 (total)
                    0.04 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)
